In [1]:
import asyncio

In [11]:
async def task_one():
    print("==Task One: Start")
    await asyncio.sleep(5)  # Simulating a non-blocking operation
    print("==Task One: End")

async def task_two():
    print("Task Two: Start")
    await asyncio.sleep(1)  # Simulating a non-blocking operation
    print("Task Two: End")

async def main():
    print("Main: Start")

    # Using asyncio.gather to run both tasks concurrently
    await asyncio.gather(task_one(), task_two())

    print("Main: End")

In [12]:
# Run the event loop to execute the asynchronous tasks
await main()

Main: Start
==Task One: Start
Task Two: Start
Task Two: End
==Task One: End
Main: End


In [13]:
async def fetch_data(url):
    print(f"Start fetching data from {url}")
    # Simulate a delay (representing network latency)
    await asyncio.sleep(2)
    print(f"Finished fetching data from {url}")
    return f"Data from {url}"

async def main():
    # Using asyncio.gather to run multiple asynchronous tasks concurrently
    results = await asyncio.gather(
        fetch_data("https://example.com/data1"),
        fetch_data("https://example.com/data2"),
        fetch_data("https://example.com/data3")
    )
    
    print("\nResults:")
    for result in results:
        print(result)

# Run the event loop to execute the asynchronous tasks
await main()

Start fetching data from https://example.com/data1
Start fetching data from https://example.com/data2
Start fetching data from https://example.com/data3
Finished fetching data from https://example.com/data1
Finished fetching data from https://example.com/data2
Finished fetching data from https://example.com/data3

Results:
Data from https://example.com/data1
Data from https://example.com/data2
Data from https://example.com/data3


In [35]:
import asyncio

async def download_csv(file_name):
    '''
    This has nothing to do with loading the CSV data via ex. pandas.
    All this does is fetching the CSV file using some Python code.
    That is why this is an I/O bounded task.
    '''
    # Simulate downloading a CSV (replace with actual download logic)
    print(f"Downloading {file_name}...")
    await asyncio.sleep(2)  # This just imitates the time it takes to download a file. 
                            # Introduces a delay of 2 seconds (because that is what sleep is doing, delaying.)
                            # We start at t=0 and after 2 second are passed (the file is downloaded) then 
                            # we can do a preprocessing on the dataset, which also takes different time
                            # and hence is also an asynchronous function and we need to await it.
                            # Same story applies to fetch_and_process_csv which makes more clear of
                            # what should be expected first (downloading) and then preprocessing right after
                            # The main is also an asynchronous functiontion because we can possibly do something
                            # after the main. Because main incorporates all the 'upper ladder' then it has
                            # implicit "awaiting" factor. Hene await main()
    return f"CSV data for {file_name}"

async def process_csv(file_name, csv_data):
    """
    So, while the downloading of the CSV data (await download_csv()) is an I/O-bound task, 
    the subsequent processing of the downloaded data (# Process the downloaded CSV data...) is where 
    CPU-bound operations may occur. If the processing logic involves heavy computations and you want 
    to take full advantage of multiple CPU cores, you might consider optimizing the CPU-bound portion.
    Techniques like parallelization or using libraries optimized for numerical computations (e.g., NumPy) 
    could be explored depending on the nature of your processing tasks.
    """
    # Process the downloaded CSV data (replace with actual processing logic)
    print(f"Processing CSV {file_name}: {csv_data}")

async def fetch_and_process_csv(file_name):
    '''
    This function just makes some logic of proper 'order' of which download and process should take place.
    '''
    # Run download_csv and process_csv concurrently for a single CSV file
    csv_data = await download_csv(file_name)
    await process_csv(file_name, csv_data)

async def main():
    # List of CSV files to fetch and process
    csv_files = ["file1.csv", "file2.csv", "file3.csv"]

    # Create tasks for fetching and processing each CSV file concurrently
    tasks = [fetch_and_process_csv(file) for file in csv_files]

    # Run tasks concurrently using asyncio.gather
    await asyncio.gather(*tasks)

await main()

Processing CSV file1.csv: CSV data for file1.csv
Processing CSV file2.csv: CSV data for file2.csv
Processing CSV file3.csv: CSV data for file3.csv
